In [ ]:
from project_sparse import *
from scipy import sparse

n = 1000
our = SparseMatrix(matrix)
sci = sparse.csr_matrix(matrix)

matrices = [randmatrix(random.random(),
random.randint(3,30),
random.randint(3,30)) for i in range(n)]

matrices2 = [randmatrix(random.random(),
matrices[i].shape[0],
matrices[i].shape[1]) for i in range(n)]

vectors = []
for i in range(n):
    vectors.append(*randmatrix(random.random(),
    1,
    matrices[i].shape[1]))

our = [SparseMatrix(matrices[i]) for i in range(n)]
sci = [sparse.csr_matrix(matrices[i]) for i in range(n)]
our2 = [SparseMatrix(matrices2[i]) for i in range(n)]
sci2 = [sparse.csr_matrix(matrices2[i]) for i in range(n)]

Changing an element present in a CSR matrix

In [ ]:
%timeit [our[i].change(1,2,25) for i in range(n)]
%timeit [sci[i].__setitem__((1,2),25) for i in range(n)]

Summing up two CSR matrices

In [ ]:
%timeit [our[i].add(our2[i]) for i in range(100)]
%timeit [sci[i] + sci2[i] for i in range(100)]

Multiplying a CSR matrix with a vector

In [ ]:
%timeit [our[i].multiply(vectors[i]) for i in range(100)]
%timeit [sci[i].multiply(vectors[i]) for i in range(100)]